# Машинное обучение, ФКН ВШЭ

## Практическое задание 8. Метод опорных векторов и аппроксимация ядер

### Общая информация
Дата выдачи: 1.10.2021

Мягкий дедлайн: 17.10.2021 23:59 МСК

Жесткий дедлайн: 24.10.2021 23:59 МСК (1 неделя -- минус балл)

### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимальная оценка за работу (без учёта бонусов) — 10 баллов.

Сдавать задание после указанного жёсткого срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.

### Формат сдачи
Загрузите решение в свой репозиторий на github и поделитесь [ссылкой на решение в форме](https://forms.gle/ZzCaqRj6bmfpSpyL7). Не забудьте дать доступ к Вашему репозиторию, что у преподавателей была возмоожность проверить работу.

### О задании

На занятиях мы подробно обсуждали метод опорных векторов (SVM). В базовой версии в нём нет чего-то особенного — мы всего лишь используем специальную функцию потерь, которая не требует устремлять отступы к бесконечности; ей достаточно, чтобы отступы были не меньше +1. Затем мы узнали, что SVM можно переписать в двойственном виде, который, позволяет заменить скалярные произведения объектов на ядра. Это будет соответствовать построению модели в новом пространстве более высокой размерности, координаты которого представляют собой нелинейные модификации исходных признаков.

Ядровой SVM, к сожалению, довольно затратен по памяти (нужно хранить матрицу Грама размера $d \times d$) и по времени (нужно решать задачу условной оптимизации с квадратичной функцией, а это не очень быстро). Мы обсуждали, что есть способы посчитать новые признаки $\tilde \varphi(x)$ на основе исходных так, что скалярные произведения этих новых $\langle \tilde \varphi(x), \tilde \varphi(z) \rangle$ приближают ядро $K(x, z)$.

Мы будем исследовать аппроксимации методом Random Fourier Features (RFF, также в литературе встречается название Random Kitchen Sinks) для гауссовых ядер. Будем использовать формулы, которые немного отличаются от того, что было на лекциях (мы добавим сдвиги внутрь тригонометрических функций и будем использовать только косинусы, потому что с нужным сдвигом косинус превратится в синус):
$$\tilde \varphi(x) = (
\cos (w_1^T x + b_1),
\dots,
\cos (w_n^T x + b_n)
),$$
где $w_j \sim \mathcal{N}(0, 1/\sigma^2)$, $b_j \sim U[-\pi, \pi]$.

На новых признаках $\tilde \varphi(x)$ мы будем строить любую линейную модель.

Можно считать, что это некоторая новая парадигма построения сложных моделей. Можно направленно искать сложные нелинейные закономерности в данных с помощью градиентного бустинга или нейронных сетей, а можно просто нагенерировать большое количество случайных нелинейных признаков и надеяться, что быстрая и простая модель (то есть линейная) сможет показать на них хорошее качество. В этом задании мы изучим, насколько работоспособна такая идея.

### Алгоритм

Вам потребуется реализовать следующий алгоритм:
1. Понизить размерность выборки до new_dim с помощью метода главных компонент.
2. Для полученной выборки оценить гиперпараметр $\sigma^2$ с помощью эвристики (рекомендуем считать медиану не по всем парам объектов, а по случайному подмножеству из где-то миллиона пар объектов): $$\sigma^2 = \text{median}_{i, j = 1, \dots, \ell, i \neq j} \left\{\sum_{k = 1}^{d} (x_{ik} - x_{jk})^2 \right\}$$
3. Сгенерировать n_features наборов весов $w_j$ и сдвигов $b_j$.
4. Сформировать n_features новых признаков по формулам, приведённым выше.
5. Обучить линейную модель (логистическую регрессию или SVM) на новых признаках.
6. Повторить преобразования (PCA, формирование новых признаков) к тестовой выборке и применить модель.

Тестировать алгоритм мы будем на данных Fashion MNIST. Ниже код для их загрузки и подготовки.

In [ ]:
import keras
from keras.datasets import fashion_mnist
(x_train_pics, y_train), (x_test_pics, y_test) = fashion_mnist.load_data()
x_train = x_train_pics.reshape(x_train_pics.shape[0], -1)
x_test = x_test_pics.reshape(x_test_pics.shape[0], -1)

4431872/4422102 [==============================] - 0s 0us/step


__Задание 1. (5 баллов)__

Реализуйте алгоритм, описанный выше. Можете воспользоваться шаблоном класса ниже или написать свой интерфейс.

Ваша реализация должна поддерживать следующие опции:
1. Возможность задавать значения гиперпараметров new_dim (по умолчанию 50) и n_features (по умолчанию 1000).
2. Возможность включать или выключать предварительное понижение размерности с помощью метода главных компонент.
3. Возможность выбирать тип линейной модели (логистическая регрессия или SVM с линейным ядром).

Протестируйте на данных Fashion MNIST, сформированных кодом выше. Если на тесте у вас получилась доля верных ответов не ниже 0.84 с гиперпараметрами по умолчанию, то вы всё сделали правильно.

In [ ]:
!pip install catboost

     |████████████████████████████████| 76.4 MB 1.3 MB/s 


In [ ]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC
import catboost


class RFFPipeline(BaseEstimator, TransformerMixin):
    def __init__(self, n_features=1000, new_dim=50, use_PCA=True, classifier='logreg'):
        """        
        Implements pipeline, which consists of PCA decomposition,
        Random Fourier Features approximation and linear classification model.
        
        n_features, int: amount of synthetic random features generated with RFF approximation.

        new_dim, int: PCA output size.
        
        use_PCA, bool: whether to include PCA preprocessing.
        
        classifier, string: either 'svm' or 'logreg', a linear classification model to use on top of pipeline.
        
        Feel free to edit this template for your preferences.    
        """
        self.n_features = n_features
        self.use_PCA = use_PCA
        self.new_dim = new_dim
        self.classifier = classifier
        
    def fit(self, X, y):
        """
        Fit all parts of algorithm (PCA, RFF, Classification) to training set.
        """
        # Your code here: (￣▽￣)/♫•*¨*•.¸¸♪
        if self.use_PCA:
            self.pca = PCA(n_components=self.new_dim)
            X = self.pca.fit_transform(X)

        random_element1, random_element2 = np.random.choice(range(len(X)), 1000000), \
                                           np.random.choice(range(len(X)), 1000000)
        mask = random_element1 != random_element2
        random_element1, random_element2 = X[random_element1[mask]], \
                                           X[random_element2[mask]]
        sigma = np.median(np.sum((random_element1-random_element2)**2, axis=1))
        self.w = np.random.normal(0, np.sqrt(1/sigma), (self.n_features, X.shape[1]))
        self.b = np.random.uniform(-np.pi, np.pi, self.n_features)
        new_X = np.cos(X.dot(self.w.T) + self.b)
        if self.classifier=='logreg':
            self.model = LogisticRegression()
        elif self.classifier=='svm':
            self.model = SVC(kernel='linear')
        self.model.fit(new_X, y)
        

    def predict_proba(self, X):
        """
        Apply pipeline to obtain scores for input data.
        """
        # Your code here: (￣▽￣)/♫•*¨*•.¸¸♪
        if self.use_PCA:
            X = self.pca.transform(X)
        new_X = np.cos(X.dot(self.w.T) + self.b)
        return self.model.predict_proba(new_X)
        
    def predict(self, X):
        """
        Apply pipeline to obtain discrete predictions for input data.
        """
        # Your code here: (￣▽￣)/♫•*¨*•.¸¸♪
        if self.use_PCA:
            X = self.pca.transform(X)
        new_X = np.cos(X.dot(self.w.T) + self.b)
        return self.model.predict(new_X)

In [ ]:
%%time
rff_logreg = RFFPipeline()
rff_logreg.fit(x_train, y_train)
preds_logreg = rff_logreg.predict(x_test)
print(f'Accuracy: {np.sum(preds_logreg==y_test)/len(preds_logreg)}')

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy: 0.8576
CPU times: user 1min 55s, sys: 20.3 s, total: 2min 15s
Wall time: 1min 11s


__Задание 2. (3 балла)__

Сравните подход со случайными признаками с обучением SVM на исходных признаках. Попробуйте вариант с обычным (линейным) SVM и с ядровым SVM. Ядровой SVM может очень долго обучаться, поэтому можно делать любые разумные вещи для ускорения: брать подмножество объектов из обучающей выборки, например.

Сравните подход со случайными признаками с вариантом, в котором вы понижаете размерность с помощью PCA и обучаете градиентный бустинг. Используйте одну из реализаций CatBoost/LightGBM/XGBoost, не забудьте подобрать число деревьев и длину шага.

Сделайте выводы — насколько идея со случайными признаками работает? Сравните как с точки зрения качества, так и с точки зрения скорости обучения и применения.

Сравнивать качество буду на подмножестве из 10000 наблюдений обучающей выборки, дабы ускорить обучение

In [ ]:
random_idx = np.random.choice(range(len(x_train)), 10000)

In [ ]:
%%time
lSVC = SVC(kernel='linear')
lSVC.fit(x_train[random_idx], y_train[random_idx])
preds = lSVC.predict(x_test)
print(f'Accuracy: {np.sum(preds==y_test)/len(preds)}')

Accuracy: 0.7992
CPU times: user 1min 2s, sys: 72 ms, total: 1min 2s
Wall time: 1min 2s


In [ ]:
%%time
rSVC = SVC(kernel='rbf')
rSVC.fit(x_train[random_idx], y_train[random_idx])
preds = rSVC.predict(x_test)
print(f'Accuracy: {np.sum(preds==y_test)/len(preds)}')

Accuracy: 0.851
CPU times: user 1min 10s, sys: 69.3 ms, total: 1min 10s
Wall time: 1min 9s


In [ ]:
%%time
rff_logreg = RFFPipeline()
rff_logreg.fit(x_train[random_idx], y_train[random_idx])
preds_logreg = rff_logreg.predict(x_test)
print(f'Accuracy: {np.sum(preds_logreg==y_test)/len(preds_logreg)}')

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy: 0.8461
CPU times: user 20.3 s, sys: 13.2 s, total: 33.5 s
Wall time: 17.6 s


In [ ]:
%%time
rff_svc = RFFPipeline(classifier='svm')
rff_svc.fit(x_train[random_idx], y_train[random_idx])
preds_svc = rff_svc.predict(x_test)
print(f'Accuracy: {np.sum(preds_svc==y_test)/len(preds_svc)}')

Accuracy: 0.8454
CPU times: user 1min 1s, sys: 1.77 s, total: 1min 2s
Wall time: 1min


In [ ]:
# гиперпараметры подбирала вручную
%%time
pca = PCA(n_components=50)
new_X = pca.fit_transform(x_train[random_idx])
model = catboost.CatBoostClassifier(random_state=42, verbose=False, depth=6, 
                                    l2_leaf_reg=20, n_estimators=500, learning_rate=0.02,
                                    bagging_temperature=20)
model.fit(new_X, y_train[random_idx])
preds = model.predict(pca.transform(x_test))[:, 0]
print(f'Accuracy: {np.sum(preds==y_test)/len(preds)}')

Accuracy: 0.8055
CPU times: user 2min 21s, sys: 2.06 s, total: 2min 23s
Wall time: 1min 13s


Результаты работы моделей обучала на случайном подмножестве обучающей выборки. В целом можно сказать, что использование RFF улучшает метрику качества в текущей задаче. Влияние расчета RFF на время выполнения минимально, а качество растет. 

Таким образом, использование RFF вместе с Linear SVC существенно улучшает качество модели.

__Задание 3. (2 балла)__

Проведите эксперименты:
1. Помогает ли предварительное понижение размерности с помощью PCA? 
2. Как зависит итоговое качество от n_features? Выходит ли оно на плато при росте n_features?
3. Важно ли, какую модель обучать — логистическую регрессию или SVM?

In [ ]:
# Your code here: (￣▽￣)/♫•*¨*•.¸¸♪

In [11]:
%%time
rff_svc = RFFPipeline(classifier='svm', use_PCA=False)
rff_svc.fit(x_train[random_idx], y_train[random_idx])
preds_svc = rff_svc.predict(x_test)
print(f'Accuracy без PCA: {np.sum(preds_svc==y_test)/len(preds_svc)}')

Accuracy без PCA: 0.0962
CPU times: user 12min 50s, sys: 917 ms, total: 12min 50s
Wall time: 12min 47s


In [12]:
%%time
rff_lr = RFFPipeline(use_PCA=False)
rff_lr.fit(x_train[random_idx], y_train[random_idx])
preds_lr = rff_lr.predict(x_test)
print(f'Accuracy без PCA: {np.sum(preds_lr==y_test)/len(preds_lr)}')

Accuracy без PCA: 0.1041
CPU times: user 7.67 s, sys: 2.4 s, total: 10.1 s
Wall time: 6.31 s


Если не использовать PCA перед RFF, то хорошего качества не получим. Это обусловлено тем, что с помощью PCA мы получаем максимально полезную информацию из признаков в новые признаки.

In [13]:
import warnings
warnings.filterwarnings("ignore")

In [14]:
for n in [10, 50, 100, 250, 500, 1000, 2000, 3000]:
    rff_svc = RFFPipeline(classifier='svm', n_features=n)
    rff_svc.fit(x_train[random_idx], y_train[random_idx])
    preds_svc = rff_svc.predict(x_test)
    print(f'Accuracy SVC c n_features={n}: {np.sum(preds_svc==y_test)/len(preds_svc)}')
    rff_lr = RFFPipeline(n_features=n)
    rff_lr.fit(x_train[random_idx], y_train[random_idx])
    preds_lr = rff_lr.predict(x_test)
    print(f'Accuracy LogReg c n_features={n}: {np.sum(preds_lr==y_test)/len(preds_lr)}')

Accuracy SVC c n_features=10: 0.6695
Accuracy LogReg c n_features=10: 0.6595
Accuracy SVC c n_features=50: 0.8121
Accuracy LogReg c n_features=50: 0.8024
Accuracy SVC c n_features=100: 0.8337
Accuracy LogReg c n_features=100: 0.8238
Accuracy SVC c n_features=250: 0.8438
Accuracy LogReg c n_features=250: 0.8368
Accuracy SVC c n_features=500: 0.8482
Accuracy LogReg c n_features=500: 0.8429
Accuracy SVC c n_features=1000: 0.8494
Accuracy LogReg c n_features=1000: 0.8436
Accuracy SVC c n_features=2000: 0.846
Accuracy LogReg c n_features=2000: 0.8436
Accuracy SVC c n_features=3000: 0.8457
Accuracy LogReg c n_features=3000: 0.849


Качество выходит на плато после n_features=500,1000. Но если n_features ниже 500, то качество заметно хуже. В целом вид модели особо не влияет на качество, оно примерно одинаково.

### Бонус

__Задание 4. (Максимум 2 балла)__

Как вы, должно быть, помните с курса МО-1, многие алгоритмы машинного обучения работают лучше, если признаки данных некоррелированы. Оказывается, что для RFF существует модификация, позволяющая получать ортогональные случайные признаки (Orthogonal Random Features, ORF). Об этом методе можно прочитать в [статье](https://proceedings.neurips.cc/paper/2016/file/53adaf494dc89ef7196d73636eb2451b-Paper.pdf). Реализуйте класс для вычисления ORF по аналогии с основным заданием. Обратите внимание, что ваш класс должен уметь работать со случаем n_features > new_dim (в статье есть замечание на этот счет). Проведите эксперименты, сравнивающие RFF и ORF, сделайте выводы.

In [ ]:
# Your code here: (￣▽￣)/♫•*¨*•.¸¸♪

__Задание 5. (Максимум 2 балла)__

Поэкспериментируйте с функциями для вычисления новых случайных признаков. Не обязательно использовать косинус от скалярного произведения — можно брать знак от него, хэш и т.д. Придумайте побольше вариантов для генерации признаков и проверьте, не получается ли с их помощью добиваться более высокого качества. Также можете попробовать другой классификатор поверх случайных признаков, сравните результаты.

In [ ]:
# Your code here: (￣▽￣)/♫•*¨*•.¸¸♪